***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import pytz


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu'
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

def convert_string_to_num(value):
    try:
        return int(value)
    except ValueError:
        return value

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S').dt.tz_localize(tz=pytz.timezone('Asia/Ho_Chi_Minh')))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)
inside['MaDH'] = inside['MaDH'].apply(lambda x: convert_string_to_num(x))


# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(lambda x: pd.to_datetime(x).dt.tz_localize(tz='Asia/Ho_Chi_Minh'))

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692', '224845']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting['N+'] = transporting['N0']+ pd.Timedelta(hours=12) # Kho lay tai HCM or HN
# transporting['N+'] = transporting['N0'] + pd.Timedelta(hours=32) # Kho lay tai cac tinh con lai
transporting['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn']) | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) &\
             ((data['KhoTra'].isnull() & (data['KhoLay'] != data['KhoHienTai'])) |
              (~data['KhoTra'].isnull() & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=36) # Kho giao tai HCM or HN
# r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=56) # Kho giao tai cac tinh con lai
r_transporting['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# set e-commerces column to NaN
data['ECommerces'] = 'Others'


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'ECommerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'ECommerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'ECommerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'ECommerces'] = 'Lazada'

# ToanTrinh
data['Aging_ToanTrinh'] = np.nan
ecoms_filter = data['MaKH'].isin(ecoms)
data.loc[ecoms_filter, 'Aging_ToanTrinh'] = datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - data[ecoms_filter]['ThoiGianTaoChuyenDoi']
data.loc[~ecoms_filter, 'Aging_ToanTrinh'] = datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - data[~ecoms_filter]['ThoiGianTao']
# reset the index of datafram
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data['NgayHienTai'] = datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')).strftime('%Y-%m-%d')
data.drop_duplicates(inplace=True)
data['Days_Aging'] = data['Aging'].dt.round("d").apply(lambda x: x.days)
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,TenNguoiGui,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,ECommerces,Aging_ToanTrinh,NgayHienTai,Days_Aging
0,10FJ1D4K7FY,NaN,2020-10-13 18:10:07+07:00,2020-10-14 08:00:00+07:00,2020-10-13 20:32:33+07:00,2020-10-15 08:14:47+07:00,NaT,2020-10-19 23:00:00+07:00,NaT,'Đồ gia dụng Ngọc Hằng',NaN,Huyện Tân Yên,Bắc Giang,A,Quận 4,Hồ Chí Minh,E,Bưu Cục Tân Yên-Bắc Giang,Bưu Cục Q.4-HCM,Bưu Cục Q.4-HCM,Bưu Cục Tân Yên-Bắc Giang,Giao hàng không thành công,500,10x10x10,1200291,'15/10/2020 05:14:12 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',3 Ngày,1.0,3.0,NaN,NaN,2046499.0,NaN,1148049,20UCL56773,2329.0,2471.0,2020-10-15 01:03:45+07:00,Kết thúc,Kho giao,2020-10-15 01:03:45+07:00,2020-10-20 01:03:45+07:00,44 days 11:01:00.557528,Others,50 days 17:54:38.668498,2020-12-03,44
1,10FJKXYDUL7,EKW2020101400159582,2020-10-14 14:12:04+07:00,2020-10-14 14:12:04+07:00,2020-10-15 10:10:57+07:00,2020-10-17 18:44:35+07:00,NaT,2020-10-16 23:00:00+07:00,NaT,'HCM Quận 12',NaN,Quận 12,Hồ Chí Minh,E,Quận 4,Hồ Chí Minh,E,Kho Khách hàng lớn Hồ Chí Minh 01,Bưu Cục Q.4-HCM,Bưu Cục Q.4-HCM,Bưu Cục 198 Thạnh Lộc 15-Q.12-HCM,Giao hàng không thành công,10,10x10x10,796163,'17/10/2020 13:40:33 - GHN-DFC1A4|KHÔNG LIÊN L...,'630000 note for express',1 Ngày,NaN,1.0,NaN,NaN,NaN,NaN,629481,20FWE15396,2329.0,2471.0,2020-10-15 14:15:00+07:00,Kết thúc,Kho giao,2020-10-15 14:15:00+07:00,2020-10-20 14:15:00+07:00,43 days 21:49:45.557528,Others,49 days 21:52:41.668498,2020-12-03,44
2,10FJUA1SA9D,NaN,2020-10-14 16:16:06+07:00,2020-10-15 08:00:00+07:00,2020-10-15 14:05:54+07:00,2020-10-16 14:36:33+07:00,NaT,2020-10-17 23:00:00+07:00,NaT,'Bông shop Tuệ Tĩnh',NaN,Quận Hai Bà Trưng,Hà Nội,G,Quận 4,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 01,Bưu Cục Q.4-HCM,Bưu Cục Q.4-HCM,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 01,Giao hàng không thành công,500,10x10x10,74132,'16/10/2020 08:22:28 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',2 Ngày,1.0,2.0,NaN,NaN,1927269.0,NaN,59445,20TTB99249,2329.0,2471.0,2020-10-16 14:25:01+07:00,Kết thúc,Kho giao,2020-10-16 14:25:01+07:00,2020-10-21 14:25:01+07:00,42 days 21:39:44.557528,Others,49 days 19:48:39.668498,2020-12-03,43
3,10FJSA6NXLR,INV/2020/11245,2020-10-16 11:14:14+07:00,2020-10-16 11:14:14+07:00,2020-10-16 20:06:37+07:00,2020-10-17 18:44:57+07:00,NaT,2020-10-19 23:00:00+07:00,NaT,'Shop Đồ Bộ Thảo Bé',NaN,Quận Tân Bình,Hồ Chí Minh,E,Quận 4,Hồ Chí Minh,E,Bưu Cục 400 Âu Cơ-Q.Tân Bình-HCM 01,Bưu Cục Q.4-HCM,Bưu Cục Q.4-HCM,Bưu Cục 400 Âu Cơ-Q.Tân Bình-HCM 01,Giao hàng không thành công,200,10x10x10,2545865,'17/10/2020 14:25:12 - GHN-DFC1A0|THAY ĐỔI THÔ...,'1 Quần Áo; 39 + 75',1 Ngày,1.0,1.0,NaN,NaN,210788.0,NaN,1290959,20VRW13587,2329.0,2471.0,2020-10-17 03:28:37+07:00,Kết thúc,Kho giao,2020-10-17 03:28:37+07:00,2020-10-22 03:28:37+07:00,42 days 08:36:08.557528,Others,48 days 00:50:31.668498,2020-12-03,42
4,10FJ66H5A7Q,SAPO330525475,2020-10-16 14:50:16+07:00,2020-10-16 14:50:16+07:00,2020-10-16 19:08:28+07:00,2020-10-17 18:44:57+07:00,NaT,2020-10-19 23:00:00+07:00,NaT,'Xưởng Mộc Việt',NaN,Quận 10,Hồ Chí Minh,E,Quận 4,Hồ Chí Minh,E,Bưu Cục 16A Lê Hồng Phong-Q.10-HCM 04,Bưu Cục Q.4-HCM,Bưu Cục Q.4-HCM,Bưu Cục 16A Lê Hồng Phong-Q.10-HCM 04,Giao hàng không thành công,10,10x10x10,2572521,'17/10/2020 15:38:32 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',1 Ngày,1.0,1.0,NaN,NaN,1875411.0,NaN,1328188,20VRW13587,2329.0,2471.0,2020-10-17 03:28:37+07:00,Kết thúc,Kho giao,2020-10-17 03:28:37+07:00,2020-10-22 03:28:37+07:00,42 days 08:36:08.557528,Others,47 days 21:14:29.668498,2020-12-03,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [3]:
# export data backlog to csv file
# data export
data_to_export = data[[
    'NgayHienTai',
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'ECommerces',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Days_Aging',
    'Aging_ToanTrinh',
    'TrangThaiLuanChuyen',
    'MaKien',
]]

data_to_export.to_csv('backlog.csv', encoding='utf-8-sig', index=False)


In [4]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g


# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_bot'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_to_export.loc[data_to_export['Aging'] >= pd.Timedelta(hours=0)]
# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_bot' id:181055436>

In [5]:
# read dataframe from google sheet
from df2gspread import gspread2df as g2df

df_gspread = g2df.download(gfile=SPREADSHEET_ID, wks_name=WKS_NAME, credentials=credentials, col_names=True, row_names=False)

# df_gspread

In [6]:
# Inventory report
# from df2gspread import df2gspread as df2g
data_inventory = data.loc[(data['LoaiBacklog'] == 'Kho giao') & (data['TrangThai'] != 'Đang giao hàng')]
data_inventory['LoaiXuLy'] = 'Chưa giao lại'
chuagiao_filter = data_inventory['ThoiGianGiaoLanDau'].isnull()
giaoloi_filter = ~data_inventory['GhiChuGHN'].isnull() & data_inventory['GhiChuGHN'].str.contains(datetime.today().strftime('%d/%m/%Y') + ' ')
data_inventory.loc[chuagiao_filter, 'LoaiXuLy'] = 'Chưa giao lần nào'
data_inventory.loc[giaoloi_filter, 'LoaiXuLy'] = 'Giao lỗi'

def split_day(x):
    try:
        return x.strftime('%Y-%m-%d')
    except ValueError:
        return np.nan

def split_time(x):
    try:
        return x.strftime('%H:%M')
    except ValueError:
        return np.nan


data_inventory['N_ve_kho'] = data_inventory['N0'].apply(lambda x: split_day(x))
data_inventory['H_ve_kho'] = data_inventory['N0'].apply(lambda x: split_time(x))

del data_inventory['N0']
del data_inventory['N+']
del data_inventory['Aging']
del data_inventory['NgayHienTai']
del data_inventory['GhiChu']
# data_inventory
# to csv file
# data_inventory.to_csv('rp_giao.csv', index=False, encoding='utf-8-sig')
# data_inventory.to_excel('report_giao.xlsx')

# pivot data and push to google sheet
table1 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], aggfunc='count')
table2 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], columns=['N_ve_kho'], aggfunc='count', fill_value=0)

# table1
WKS_INVENTORY = 'RPGiao_bot'
# inventory_wks = gc.open_by_key(SPREADSHEET_ID).worksheet(WKS_INVENTORY)
# # inventory_wks.clear()  # clear old data
df2g.upload(
    table1,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    start_cell='A1',
)

# inventory_wks.add_row(table2.shape[0])

df2g.upload(
    table2,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    clean=False,
    start_cell='A8',
)
# then write detail inventory data
inventory_upload = data_inventory[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'ThongTinTraHang',
    'GhiChuGHN',
    'SoLanGiao',
    'ECommerces',
    'LoaiXuLy',
    'N_ve_kho',
    'H_ve_kho',
    
]]
# export to csv
inventory_upload.to_csv('rp_giao.csv', encoding='utf-8-sig', index=False)

df2g.upload(
    inventory_upload,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=False,
    col_names=True,
    clean=False,
    start_cell='A15',
)

<Worksheet 'RPGiao_bot' id:1115912348>

In [7]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val